In [10]:
import pandas as pd
import os
from shapely.geometry import Point, Polygon
from shapely import geometry

In [14]:
directory = '../../data/processed'
void = ['clean_areas.csv', 'clean_disadvantaged_areas.csv', 'clean_police_districts.csv', 'clean_public_healthindicator.csv', 'clean_police_sentiment.csv', 'clean_ridership.csv']
data = {}

for filename in os.listdir(directory):
    if filename not in void and filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        data[filename[:-4]] = pd.read_csv(file_path)
        print(f'{filename[:-4]} successfully read in')

clean_311 successfully read in
clean_bike_stations successfully read in
clean_bike_trips successfully read in
clean_bus_stations successfully read in
clean_crime successfully read in
clean_cta_stations successfully read in
clean_police_stations successfully read in
clean_ridership successfully read in


In [13]:
for df_name, df in data.items():
    print(df_name, len(df), df.columns)

clean_311 468201 Index(['date', 'id', 'type', 'lat', 'long'], dtype='object')
clean_bike_stations 575 Index(['station', 'lat', 'long', 'id'], dtype='object')
clean_bike_trips 42485480 Index(['id', 'date', 'station_id', 'station_name'], dtype='object')
clean_bus_stations 398 Index(['stop_id', 'cta_stop_name', 'lat', 'long'], dtype='object')
clean_crime 7824317 Index(['id', 'date', 'type', 'lat', 'long'], dtype='object')
clean_cta_stations 143 Index(['station_number', 'station_name', 'lat', 'long'], dtype='object')
clean_police_sentiment 4899 Index(['district', 'sector', 'start_date', 'end_date', 'safety', 'trust',
       't_listen', 't_respect', 's_race_african_american',
       's_race_asian_american', 's_race_hispanic', 's_race_white', 's_age_low',
       's_age_medium', 's_age_high', 's_sex_female', 's_sex_male',
       's_education_low', 's_education_medium', 's_education_high',
       's_income_low', 's_income_medium', 's_income_high'],
      dtype='object')
clean_police_stations 2

In [4]:
clean_police_districts = pd.read_csv('../../data/processed/clean_police_districts.csv')

In [5]:
def parse_polygon(polygon_string):
    points = polygon_string.strip('POLYGON ((').strip('))').split(', ')
    points = [tuple(map(float, point.split())) for point in points]
    return Polygon(points)

In [6]:
clean_police_districts['geom'] = clean_police_districts['geom'].apply(parse_polygon)

In [7]:
def determine_within(df):

    statuses = []
    districts = []
    for i in range(len(df)):
        point = geometry.Point(df.lat.loc[i], df.long.loc[i])
        status = 0

        for district in clean_police_districts['geom']:
            if (district.contains(point)): 
                status = 1
                break
        statuses.append(status)
        districts.append(district)

    df['status'] = statuses
    df['district'] = districts

    return df[df['status'] == 1]

In [8]:
def determine_districts(data):
    for df_name, df in data.items():
        data[df_name] = determine_within(df)
        print(f'{df_name} successfully completed')
    return data

In [9]:
data = determine_districts(data)

clean_311 successfully completed
clean_bike_stations successfully completed


AttributeError: 'DataFrame' object has no attribute 'lat'

- Gather district geom data
- Assign district to all locational data
- Create a master dataset with all features
    - based on crime dataset
    - attach aggregated data from each other dataset based on time and district of each crime
    - attach lag and leads of crime, bike, cta, and 311 data 